In [64]:
import pathlib 
directory = pathlib.Path("./Reference/")
python_files = directory.rglob("*.py")

In [18]:
def construct_list_of_functions(raw_code):
    lines = raw_code.split('\n')
    start = -1
    functions = []
    begin_considering_function_termination = False
    amnt_tabs = 0
    for i in range(len(lines)):
        # disregard empty lines (prune trailing whitespace later)
        if (start != -1 and len(lines[i]) > 0):
            amnt_tabs_new = len(lines[i].rstrip()) - len(lines[i].strip())
            if amnt_tabs_new <= amnt_tabs and begin_considering_function_termination:
                functions.append(("\n".join(lines[start:i])).strip())
                start = -1
                begin_considering_function_termination = False
        if lines[i].lstrip().startswith(("def ", "async def ")):
            start = i
            amnt_tabs = len(lines[i].rstrip()) - len(lines[i].strip())
        if start != -1 and not begin_considering_function_termination and ":" in lines[i] and ")" in lines[i]:
            begin_considering_function_termination = True 
    return functions

In [65]:
dicty = dict()
dicty["function"] = []
dicty["intent_category"] = []

for file in python_files:
    with open(file) as f:
        parsed_path = str(file).split('/')
        f_content = f.read()
        file_without_docstrings = []
        started = False
        for line in f_content.split('\n'):
            if '"""' in line:
                if started:
                    started = False
                else:
                    started = True
                continue
            if not started:
                file_without_docstrings.append(line)
        functions = construct_list_of_functions('\n'.join(file_without_docstrings))
        if len(functions) > 0:
            dicty["function"] += functions
            dicty["intent_category"] += [parsed_path[1]] * len(functions)

In [66]:
print(len(dicty["function"]))

2768


In [67]:
from datasets import Dataset

ds = Dataset.from_dict(dicty)
ds.push_to_hub("functions_annotated_with_intents")

Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  1.93it/s]
Updating downloaded metadata with the new split.


In [37]:
# print(dicty["function"])
print(set(dicty["intent_category"]))

{'greedy_methods', 'project_euler', 'compression', 'graphs', 'divide_and_conquer', 'arithmetic_analysis', 'graphics', 'sorts', 'scripts', 'digital_image_processing', 'conversions', 'fractals', 'hashes', 'boolean_algebra', 'searches', 'bit_manipulation', 'neural_network', 'file_transfer', 'web_programming', 'knapsack', 'computer_vision', 'linear_algebra', 'blockchain', 'audio_filters', 'ciphers', 'geodesy', 'strings', 'financial', 'quantum', 'matrix', 'dynamic_programming', 'cellular_automata', 'data_structures', 'electronics', 'genetic_algorithm', 'other', 'physics', 'backtracking', 'scheduling', 'maths', 'machine_learning'}


In [68]:
from datasets import load_dataset

ds = load_dataset("michaelnath/functions_annotated_with_intents", split="train")


Using custom data configuration michaelnath--functions_annotated_with_intents-ec837b91e48cef18


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 760.53it/s]
                                                                       

Dataset parquet downloaded and prepared to /Users/michaelnath/.cache/huggingface/datasets/michaelnath___parquet/michaelnath--functions_annotated_with_intents-ec837b91e48cef18/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


In [48]:
# print(ds["function"][0])
started = False

file_without_docstrings = []
for line in ds["function"][0].split('\n'):
    if '"""' in line:
        if started:
            started = False
        else:
            started = True
        continue
    if not started:
        file_without_docstrings.append(line)
ds["function"][0] = "HI"

In [71]:
print("\n".join(ds["function"][:50]))

def bfs(graph, s, t, parent):
    # Return True if there is node that has not iterated.
    visited = [False] * len(graph)
    queue = [s]
    visited[s] = True

    while queue:
        u = queue.pop(0)
        for ind in range(len(graph[u])):
            if visited[ind] is False and graph[u][ind] > 0:
                queue.append(ind)
                visited[ind] = True
                parent[ind] = u

    return visited[t]
def mincut(graph, source, sink):
    parent = [-1] * (len(graph))
    max_flow = 0
    res = []
    temp = [i[:] for i in graph]  # Record original cut, copy.
    while bfs(graph, source, sink, parent):
        path_flow = float("Inf")
        s = sink

        while s != source:
            # Find the minimum value in select path
            path_flow = min(path_flow, graph[parent[s]][s])
            s = parent[s]

        max_flow += path_flow
        v = sink

        while v != source:
            u = parent[v]
            graph[u][v] -= path_flow
            